Harika bir seçim\! Generative Adversarial Networks (GAN), yapay zekanın en büyüleyici konularından biridir. İstediğin gibi, kodun içine boğulmadan önce **GAN'ın mantığını** çok basit bir hikaye ile anlatalım, sonra da bu hikayeyi koda dökelim.

Aşağıda, bir Jupyter Notebook gibi adım adım çalıştırabileceğin, açıklamalı ve sonunda eğitim sürecinin GIF'ini oluşturan kodu hazırladım.

-----

### 1\. GAN Nedir? (Kalpazan ve Dedektif Hikayesi)

GAN, aslında birbiriyile yarışan iki ayrı yapay zeka modelidir.

1.  **Generator (Üretici - "Kalpazan"):**
      * **Görevi:** Hiç yoktan (rastgele sayılardan) gerçekçi sahte paralar (bizim durumumuzda el yazısı rakamlar) üretmek.
      * **Amacı:** Dedektifi kandırmak. "Bak bu gerçek\!" dedirtmek.
2.  **Discriminator (Ayırt Edici - "Dedektif"):**
      * **Görevi:** Eline gelen resmin "Gerçek veri setinden mi" yoksa "Kalpazanın ürettiği sahte mi" olduğunu anlamak.
      * **Amacı:** Asla kandırılmamak.

[Image of GAN architecture diagram]

**Eğitim Süreci Şöyle İşler:**

  * Kalpazan ilk başta berbat resimler çizer (karıncalı ekran gibi).
  * Dedektif buna bakar ve "Bu sahte\!" der.
  * Kalpazan, "Neyi yanlış yaptım?" diye bakar ve kendini geliştirir.
  * Dedektif de sürekli gerçek resimler görerek kendini geliştirir.
  * Bu döngü binlerce kez tekrarlandığında, Kalpazan o kadar iyi sahteler üretir ki, Dedektif artık ayırt edemez hale gelir. İşte o an GAN eğitilmiş demektir.

-----

### 2\. Kodlama Aşaması

Bu kodu bir Jupyter Notebook hücresi gibi veya `.py` dosyası olarak çalıştırabilirsin.

#### Adım 1: Kütüphaneler ve Veri Hazırlığı

Veriyi **-1 ile 1 arasına** sıkıştırıyoruz. Bu GAN'lar için çok önemlidir çünkü `tanh` aktivasyon fonksiyonunu kullanacağız.

In [6]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import imageio
import os
import glob

# 1. Veri Setini Yükle ve Hazırla
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

# Görüntüleri [28, 28, 1] boyutuna getir (Siyah beyaz kanalını ekle)
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')

# Pikselleri -1 ile 1 arasına normalize et (GAN standardı)
# Normalde pikseller 0-255 arasındadır.
train_images = (train_images - 127.5) / 127.5

# Veriyi karıştır ve batch'lere böl
BUFFER_SIZE = 10000
BATCH_SIZE = 256
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

print("Veri hazır! Görüntü boyutu:", train_images.shape)

Veri hazır! Görüntü boyutu: (60000, 28, 28, 1)


#### Adım 2: Modelleri Oluşturma (Vanilla GAN)

Burada Convolution (CNN) yerine `Dense` (Tam Bağlantılı) katmanlar kullanarak en temel "Vanilla" yapısını kuruyoruz.

In [3]:
# --- GENERATOR (ÜRETİCİ / KALPAZAN) ---
def make_generator_model():
    model = tf.keras.Sequential()
    # Giriş: Rastgele gürültü vektörü (seed). Boyutu 100 olsun.
    # Bu gürültüyü alıp 7*7*256 boyutunda bir veriye dönüştüreceğiz.
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU()) # ReLU yerine LeakyReLU GAN'da daha iyi çalışır

    # Şeklini bir görüntü taslağına benzetelim
    model.add(layers.Reshape((7, 7, 256)))

    # Upsampling (Görüntüyü büyütme) - 7x7 -> 14x14
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # 14x14 -> 28x28 (Final boyutu)
    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Çıktı katmanı: 28x28x1 boyutunda görüntü.
    # Aktivasyon 'tanh' çünkü veriyi -1 ile 1 arasına sıkıştırmıştık.
    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

    return model

# --- DISCRIMINATOR (AYIRT EDİCİ / DEDEKTİF) ---
def make_discriminator_model():
    model = tf.keras.Sequential()
    # Giriş: 28x28x1 boyutunda bir resim
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]))
    model.add(layers.LeakyReLU())

    model.add(layers.Dropout(0.3)) # Ezberlemeyi önlemek için

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten()) # Düzleştir
    model.add(layers.Dense(1))  # Çıktı: Tek bir sayı (Gerçek olma ihtimali)

    return model

generator = make_generator_model()
discriminator = make_discriminator_model()

print("Modeller oluşturuldu.")

Modeller oluşturuldu.


#### Adım 3: Kayıp Fonksiyonu ve Optimizasyon

Dedektif ve Kalpazan'ın ne kadar başarılı olduğunu nasıl ölçeceğiz?

In [4]:
# Binary Crossentropy: İki sınıflı (Gerçek/Sahte) sınıflandırma için standart.
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# --- Discriminator Kaybı ---
# Dedektif şunları yapmalı:
# 1. Gerçek resimlere "1" (Gerçek) demeli.
# 2. Sahte resimlere "0" (Sahte) demeli.
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output) # Gerçekleri 1 bildi mi?
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output) # Sahteleri 0 bildi mi?
    return real_loss + fake_loss

# --- Generator Kaybı ---
# Kalpazan şunu yapmalı:
# Ürettiği sahte resimlere Dedektifin "1" (Gerçek) demesini sağlamalı.
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Optimizasyoncular (Adam optimizer standarttır)
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

#### Adım 4: Eğitim Döngüsü (En Kritik Yer)

Burası sihrin gerçekleştiği yer. Her adımda:

1.  Generator bir resim üretir.
2.  Discriminator hem gerçek hem sahte resme bakar.
3.  İkisi de hatalarına göre güncellenir.

<!-- end list -->

In [7]:
EPOCHS = 50 # Eğitim tur sayısı
noise_dim = 100 # Rastgele gürültü boyutu
num_examples_to_generate = 16 # GIF için her seferinde 16 örnek kaydedelim

# Sabit bir gürültü (seed) oluşturuyoruz ki eğitimin gelişimini hep aynı "tohumdan" görelim.
seed = tf.random.normal([num_examples_to_generate, noise_dim])

@tf.function # Bu dekoratör işlemi hızlandırır
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # 1. Üretici sahte resim üretir
        generated_images = generator(noise, training=True)

        # 2. Dedektif hem gerçek hem sahte resimleri değerlendirir
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        # 3. Kayıplar hesaplanır
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    # 4. Gradyanlar (değişim yönü) hesaplanır
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    # 5. Modeller güncellenir (Öğrenme gerçekleşir)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

# Görselleştirme Fonksiyonu
def generate_and_save_images(model, epoch, test_input):
    # Eğitim değil (training=False), sadece çıktı alıyoruz
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        # Görüntüyü -1,1 aralığından 0,255 aralığına çekip gösteriyoruz
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')

    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.close() # Belleği şişirmemek için kapat
    # (Eğer notebook'ta anlık görmek istersen plt.show() diyebilirsin ama GIF için savefig şart)

# --- EĞİTİMİ BAŞLAT ---
print("Eğitim başlıyor... Lütfen bekleyin (GPU yoksa biraz sürebilir)")

for epoch in range(EPOCHS):
    for image_batch in train_dataset:
        train_step(image_batch)

    # Her epoch sonunda ilerlemeyi kaydet
    print(f"Epoch {epoch + 1} tamamlandı.")
    generate_and_save_images(generator, epoch + 1, seed)

print("Eğitim Tamamlandı!")

Eğitim başlıyor... Lütfen bekleyin (GPU yoksa biraz sürebilir)


2025-11-24 04:03:17.193067: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1 tamamlandı.


2025-11-24 04:05:52.209858: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 2 tamamlandı.
Epoch 3 tamamlandı.


2025-11-24 04:10:49.840860: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 4 tamamlandı.
Epoch 5 tamamlandı.
Epoch 6 tamamlandı.
Epoch 7 tamamlandı.


2025-11-24 04:20:51.332746: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 8 tamamlandı.
Epoch 9 tamamlandı.
Epoch 10 tamamlandı.
Epoch 11 tamamlandı.
Epoch 12 tamamlandı.


KeyboardInterrupt: 

#### Adım 5: GIF Oluşturma

Eğitim bittiğinde klasörde bir sürü `image_at_epoch_xxxx.png` olacak. Bunları birleştirip süreci izleyelim.

In [ ]:
anim_file = 'gan_egitimi.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
    filenames = glob.glob('image_at_epoch*.png')
    filenames = sorted(filenames)
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
    image = imageio.imread(filename)
    writer.append_data(image)

print(f"GIF oluşturuldu: {anim_file}")

# Sonuçlardan birini ekrana bas
import IPython
if os.path.exists(anim_file):
    print("GIF hazırlanıyor...")
    # Colab veya Jupyter kullanıyorsan:
    # IPython.display.Image(open(anim_file,'rb').read())
else:
    print("GIF dosyası bulunamadı.")

-----

### Çıktıda Ne Göreceksin?

1.  **İlk Epochlar:** Gri, anlamsız gürültü kareleri. (Kalpazan henüz işi bilmiyor).
2.  **Orta Epochlar:** Mürekkep lekeleri gibi şekiller, yavaş yavaş rakama benzeyen karaltılar.
3.  **Son Epochlar:** 0'dan 9'a kadar belirgin, el yazısı rakamlar.

Bu kodda `make_generator_model` içinde `Conv2DTranspose` kullandım. Eğer "en basit Vanilla GAN" (sadece Dense katmanlar) istersen, Convolution katmanlarını silip sadece `Dense` katmanlar koyabiliriz ama o zaman görüntüler çok daha bulanık olur. Bu haliyle (DCGAN türevi) sonuçlar çok daha tatmin edici olacaktır.

**Bir sonraki adım:** Bu kodu Google Colab'de "GPU" seçeneğini açarak çalıştırmanı öneririm, CPU'da 50 epoch biraz uzun sürebilir. Yardıma ihtiyacın olursa buradayım\!